# Rule: **simplify_network**

**Outputs**

- resources/networks/`base_s.nc`
- resources/`regions_onshore_base_s.geojson`
- resources/`regions_offshore_base_s.geojson`
- resources/`busmap_base_s.csv`

In [ ]:
######################################## Parameters

### Run
name = ''
prefix = ''

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import numpy as np


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_base_s.geojson'
file_regions_offshore = f'regions_offshore_base_s.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)

## `base_s.nc`

Load the network and show its components.

In [ ]:
file = f'base_s.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3
link_widths = 1*n.links.p_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, 
       line_widths=line_widths, 
       link_widths=link_widths, 
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.buses`

Place `n.buses` in a dataFrame and show its content.

In [ ]:
bb = n.buses

bb.head()

### Variable: `n.carriers`

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.lines`

Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the distribution of line lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(ln['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.links`

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

Place DC links in a dataFrame and show its content.

In [ ]:
lk_DC = lk.loc[lk['carrier']=='DC']

lk_DC.head()

How is the distribution of DC link lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(lk_DC['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.shapes`

Place `n.shapes` in a geodataFrame and show its content.

In [ ]:
sh = n.shapes

sh.head()

Represent the shapes.

In [ ]:
sh.plot()

## `regions_onshore_base_s.geojson`

Show its content.

In [ ]:
gdf_regions_onshore.head()

How many regions are there in the file?

In [ ]:
len(gdf_regions_onshore)

In [ ]:
gdf_regions_onshore.plot()

## `regions_offshore_base_s.geojson`

Show its content.

In [ ]:
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)

gdf_regions_offshore.head()

How many regions are there in the file?

In [ ]:
len(gdf_regions_offshore)

In [ ]:
gdf_regions_offshore.plot()